In [ ]:
import numpy as np
import pandas as pd 
import logging

from gensim.models import Word2Vec


from words_parser import ingredient_parser

In [ ]:
 
data = pd.read_csv(r'C:\xampp\htdocs\3161Database files\recipe_recommender\app\csvfiles\parsed_recipesv4.csv')
data['parsed_ingredients'] = data.RecipeIngredientParts.apply(ingredient_parser)
data.head()

In [ ]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_ingr(data):
    ingreds_sorted = []
    for doc in data.parsed_ingredients.values:
        ingreds_sorted.append(doc)
    return ingreds_sorted

ingreds = get_and_sort_ingr(data[:5000])
#for i in ingreds:
#    print(i)
print(f"Length of ingreds: {len(ingreds)}")

In [ ]:
thirdofcorpus = round(len(ingreds)*.25)
#print(len(ingreds))
print(len(ingreds[:thirdofcorpus]))

lengths = [len(doc) for doc in ingreds]
print(round(sum(lengths)/len(lengths)))
avg_len = float(sum(lengths)) / len(lengths)
avg_len


In [ ]:
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 8 # number of CPUs
window = 6 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(ingreds, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [ ]:
#Summarize the loaded model
print(model_cbow)

#Summarize vocabulary
words = list(model_cbow.wv.index_to_key)
words.sort()
print(words)

#Acess vector for one word
print("check ",model_cbow.wv['tomato'])

# most similar
model_cbow.wv.most_similar(u'tomato')
#print("after stock")
#model_cbow.wv.similarity('cauliflower', 'cauliflower just larger than potato')

In [ ]:
model_cbow.save('models/model_cbow_ingredients.bin')

In [ ]:
class MeanEmbeddingVectorizer(object):

	def __init__(self, word_model):
		self.word_model = word_model
		self.vector_size = word_model.wv.vector_size

	def fit(self):  # comply with scikit-learn transformer requirement
		return self

	def transform(self, docs):  # comply with scikit-learn transformer requirement
		doc_word_vector = self.word_average_list(docs)
		return doc_word_vector

	def word_average(self, sent):
		"""
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""
		mean = []
		for word in sent:
			if word in self.word_model.wv.index_to_key:
				mean.append(self.word_model.wv.get_vector(word))

		if not mean:  # empty words
			# If a text is empty, return a vector of zeros.
			logging.warning("cannot compute average owing to no vector for {}".format(sent))
			return np.zeros(self.vector_size)
		else:
			mean = np.array(mean).mean(axis=0)
			return mean


	def word_average_list(self, docs):
		"""
		Compute average word vector for multiple docs, where docs had been tokenized.
		:param docs: list of sentence in list of separated tokens
		:return:
			array of average word vector in shape (len(docs),)
		"""
		return np.vstack([self.word_average(sent) for sent in docs])

In [ ]:
# encode document by averaging word embeddings

# load model 
loaded_model = Word2Vec.load('models/model_cbow_ingredients.bin')
if loaded_model:
    print("Successfully loaded model")

mean_vec_tr = MeanEmbeddingVectorizer(loaded_model)
doc_vec = mean_vec_tr.transform(ingreds)